In [14]:
%matplotlib inline
import afgiftspligtig_værdi as av
import matplotlib.patches as mpatches
import logit
import clogit_estimation as cle
import pandas as pd
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
import requests as rq
from io import BytesIO
import matplotlib as mpl

In [15]:
path = "https://raw.githubusercontent.com/Jona327a/Bachelor-projekt/main/Data/choice_data_subset.xlsx"

In [16]:
def data_set(path):
    data = rq.get(path).content
    choice_data_subset = pd.read_excel(BytesIO(data))
    return choice_data_subset

In [17]:
def theta_hat():
    choice_data_subset = data_set(path)
    
    choice_data_subset['Prices (2015-DKK)'] = np.log(choice_data_subset['Prices (2015-DKK)'] / 100000)
    choice_data_subset['Weight (kg)'] = choice_data_subset['Weight (kg)'] / 1000
    choice_data_subset['Engine effect (kW)'] = choice_data_subset['Engine effect (kW)'] / 100
    
    variables = ['Prices (2015-DKK)', 'Weight (kg)', 'Engine effect (kW)', 'Size (m3)', 'Cost/km (DKK)']
    N = choice_data_subset.Year.nunique()
    J = choice_data_subset[['Year', 'key']].groupby('Year').count().min().tolist()[0]
    x, y, x_vars = cle.get_x_y(dataset = choice_data_subset, N = N, J = J,  dummyvar = 'Fuel', attributes = variables)
    
    result = cle.clogt_estimation(x = x, y = y, x_vars = x_vars)
    theta_hat = result['theta_hat']
    return theta_hat

In [18]:
def new_prices(rate_1, rate_2, rate_3, reg_beløb_1, reg_beløb_2, bundfradrag, el_bundfradrag_1, el_fradrag_2, indfasning):
    choice_data_subset = data_set(path)
    
    new_prices = []
    for i in range(0, choice_data_subset.shape[0]):
        av_i = choice_data_subset.loc[i, 'afgiftspligtig_værdi']
        fuel_i = choice_data_subset.loc[i, 'Fuel']
        if fuel_i == 'El':
            new_price = av.pris_efter_reg_afgift_el(afgiftspligtig_værdi = av_i, bundfradrag = bundfradrag, ekstra_bundfradrag = el_bundfradrag_1, indfasning = indfasning, 
                                                    reg_beloeb1 = reg_beløb_1, reg_beloeb2 = reg_beløb_2, el_fradrag = el_fradrag_2,
                                                    part_1 = rate_1, part_2 = rate_2, part_3 = rate_3)
            new_prices.append(new_price)
        elif fuel_i == 'Benzin':
            new_price = av.pris_efter_reg_afgift_ben(afgiftspligtig_værdi = av_i, bundfradrag = bundfradrag, 
                                                    reg_beloeb1 = reg_beløb_1, reg_beloeb2 = reg_beløb_2, 
                                                    part_1 = rate_1, part_2 = rate_2, part_3 = rate_3)
            new_prices.append(new_price)
        else:
            new_price = av.pris_efter_reg_afgift_die(afgiftspligtig_værdi = av_i, bundfradrag = bundfradrag, 
                                                    reg_beloeb1 = reg_beløb_1, reg_beloeb2 = reg_beløb_2,
                                                    part_1 = rate_1, part_2 = rate_2, part_3 = rate_3)
            new_prices.append(new_price)
    return new_prices

In [19]:
def interactive_divided_cp(rate_1 = 0.25, rate_2 = 0.85, rate_3 = 1.5, reg_beløb_1 = 65000, reg_beløb_2 = 202200, bundfradrag = 21700, el_bundfradrag_1 = 170000, el_fradrag_2 = 1700, indfasning = 0.40):
    
    choice_data_subset = data_set(path)
    
    choice_data_subset['Prices (2015-DKK)'] = np.log(choice_data_subset['Prices (2015-DKK)'] / 100000)
    choice_data_subset['Weight (kg)'] = choice_data_subset['Weight (kg)'] / 1000
    choice_data_subset['Engine effect (kW)'] = choice_data_subset['Engine effect (kW)'] / 100

    variables = ['Prices (2015-DKK)', 'Weight (kg)', 'Engine effect (kW)', 'Size (m3)', 'Cost/km (DKK)']
    N = choice_data_subset.Year.nunique()
    J = choice_data_subset[['Year', 'key']].groupby('Year').count().min().tolist()[0]
    x, y, x_vars = cle.get_x_y(dataset = choice_data_subset, N = N, J = J,  dummyvar = 'Fuel', attributes = variables)

    result = cle.clogt_estimation(x = x, y = y, x_vars = x_vars)
    theta_hat = result['theta_hat']
    
    # CP for de originale priser:
    cp_original_prices = logit.choice_prob(theta_hat, x)
    cp_original_prices = cp_original_prices.reshape(N*J,)
    choice_data_subset = choice_data_subset.assign(cp_original_prices = cp_original_prices)
    cp_original_prices_data = choice_data_subset[['Year', 'cp_original_prices', 'Fuel']].set_index(['Fuel'])
    cp_original_prices_el_data = cp_original_prices_data.loc['El']
    cp_original_prices_el_sum = cp_original_prices_el_data.groupby('Year')['cp_original_prices'].sum()
    cp_original_prices_benzin_data = cp_original_prices_data.loc['Benzin']
    cp_original_prices_benzin_sum = cp_original_prices_benzin_data.groupby('Year')['cp_original_prices'].sum()
    cp_original_prices_diesel_data = cp_original_prices_data.loc['Diesel']
    cp_original_prices_diesel_sum = cp_original_prices_diesel_data.groupby('Year')['cp_original_prices'].sum()
    
    # CP for new_prices:
    choice_data_subset = choice_data_subset.assign(new_prices = new_prices(rate_1, rate_2, rate_3, reg_beløb_1, reg_beløb_2, bundfradrag, el_bundfradrag_1, el_fradrag_2, indfasning))
    choice_data_subset['new_prices'] = np.log(choice_data_subset['new_prices'] / 100000)
    variables_v2 = ['new_prices', 'Weight (kg)', 'Engine effect (kW)', 'Size (m3)', 'Cost/km (DKK)']
    x_v2, y_v2, x_vars_v2 = cle.get_x_y(dataset = choice_data_subset, N = N, J = J,  dummyvar = 'Fuel', attributes = variables_v2)
    
    cp_new_prices = logit.choice_prob(theta_hat, x_v2)
    cp_new_prices = cp_new_prices.reshape(N*J,)
    choice_data_subset = choice_data_subset.assign(cp_new_prices = cp_new_prices)
    cp_new_prices_data = choice_data_subset[['Year', 'cp_new_prices', 'Fuel']].set_index(['Fuel'])
    cp_new_prices_el_data = cp_new_prices_data.loc['El']
    cp_new_prices_el_sum = cp_new_prices_el_data.groupby('Year')['cp_new_prices'].sum()
    cp_new_prices_benzin_data = cp_new_prices_data.loc['Benzin']
    cp_new_prices_benzin_sum = cp_new_prices_benzin_data.groupby('Year')['cp_new_prices'].sum()
    cp_new_prices_diesel_data = cp_new_prices_data.loc['Diesel']
    cp_new_prices_diesel_sum = cp_new_prices_diesel_data.groupby('Year')['cp_new_prices'].sum()
    
    print("\nSummen af valgsandsynlighederne for elbiler i 2020:")
    print("For originale priser =", round(cp_original_prices_el_sum[2020], 6))
    print("For nye priser =", round(cp_new_prices_el_sum[2020], 6))
    
    print("\nSummen af valgsandsynlighederne for benzinbiler i 2020:")
    print("For originale priser =", round(cp_original_prices_benzin_sum[2020], 6))
    print("For nye priser =", round(cp_new_prices_benzin_sum[2020], 6))
    
    print("\nSummen af valgsandsynlighederne for dieselbiler i 2020:")
    print("For originale priser =", round(cp_original_prices_diesel_sum[2020], 6))
    print("For nye priser =", round(cp_new_prices_diesel_sum[2020], 6))
    
    fig, (ax_el, ax_benzin, ax_diesel) = plt.subplots(1, 3)
    
    # Plotting el
    ax_el.set_xlabel('År')
    ax_el.set_ylabel('Valgsandsynlighed', color = 'black') 
    line_el_1, = ax_el.plot(cp_new_prices_el_data['Year'].drop_duplicates(), cp_new_prices_el_sum, color = 'y')
    ax_el.grid()
    
    line_el_2, = ax_el.plot(cp_original_prices_el_data['Year'].drop_duplicates(), cp_original_prices_el_sum, color = 'g', linestyle = 'dashed')
    
    el_nye_priser = mpatches.Patch(color = 'y', label = 'Nye priser')
    el_originale_priser = mpatches.Patch(color = 'g', label = 'Originale priser')
    ax_el.legend(handles = [el_nye_priser, el_originale_priser], loc = 2)
    ax_el.set_title("Forudsagt efterspørgsel for elbiler")
    
    line_el_1.set_ydata(cp_new_prices_el_sum)
    line_el_2.set_ydata(cp_original_prices_el_sum)
    fig.canvas.draw_idle()
    
    # Plotting benzin
    ax_benzin.set_xlabel('År')
    #ax_benzin.set_ylabel('Valgsandsynlighed', color = 'black') 
    line_benzin_1, = ax_benzin.plot(cp_new_prices_benzin_data['Year'].drop_duplicates(), cp_new_prices_benzin_sum, color = 'y')
    ax_benzin.grid()
    
    line_benzin_2, = ax_benzin.plot(cp_original_prices_benzin_data['Year'].drop_duplicates(), cp_original_prices_benzin_sum, color = 'r', linestyle = 'dashed')

    benzin_nye_priser = mpatches.Patch(color = 'y', label = 'Nye priser')
    benzin_originale_priser = mpatches.Patch(color = 'r', label = 'Originale priser')
    ax_benzin.legend(handles = [benzin_nye_priser, benzin_originale_priser], loc = 2)
    ax_benzin.set_title("Forudsagt efterspørgsel for benzinbiler")

    line_benzin_1.set_ydata(cp_new_prices_benzin_sum)
    line_benzin_2.set_ydata(cp_original_prices_benzin_sum)
    fig.canvas.draw_idle()
    
    # Plotting diesel
    ax_diesel.set_xlabel('År')
    #ax_diesel.set_ylabel('Valgsandsynlighed', color = 'black') 
    line_diesel_1, = ax_diesel.plot(cp_new_prices_diesel_data['Year'].drop_duplicates(), cp_new_prices_diesel_sum, color = 'y')
    ax_diesel.grid()
    
    line_diesel_2, = ax_diesel.plot(cp_original_prices_diesel_data['Year'].drop_duplicates(), cp_original_prices_diesel_sum, color = 'b', linestyle = 'dashed')

    diesel_nye_priser = mpatches.Patch(color = 'y', label = 'Nye priser')
    diesel_originale_priser = mpatches.Patch(color = 'b', label = 'Originale priser')
    ax_diesel.legend(handles = [diesel_nye_priser, diesel_originale_priser], loc = 3)
    ax_diesel.set_title("Forudsagt efterspørgsel for dieselbiler")

    line_diesel_1.set_ydata(cp_new_prices_diesel_sum)
    line_diesel_2.set_ydata(cp_original_prices_diesel_sum)
    fig.canvas.draw_idle()
    
#interact(interactive_divided_cp, rate_1 = (0.0, 0.5, 0.01), rate_2 = (0.0, 1.7, 0.01), rate_3 = (0.0, 3.0, 0.01), reg_beløb_1 = (0.0, 77100.0, 100.0), reg_beløb_2 = (0.0, 240000.0, 100.0), bundfradrag = (0.0, 25700.0, 100.0), el_bundfradrag_1 = (137000.0, 170000.0, 100.0), el_fradrag_2 = (0.0, 1700.0, 100.0), indfasning = (0.0, 1.0, 0.1));


In [20]:
def interactive_cp(rate_1 = 0.25, rate_2 = 0.85, rate_3 = 1.5, reg_beløb_1 = 65000, reg_beløb_2 = 202200, bundfradrag = 21700, el_bundfradrag_1 = 170000, el_fradrag_2 = 1700, indfasning = 0.40):
    
    choice_data_subset = data_set(path)
    
    choice_data_subset['Prices (2015-DKK)'] = np.log(choice_data_subset['Prices (2015-DKK)'] / 100000)
    choice_data_subset['Weight (kg)'] = choice_data_subset['Weight (kg)'] / 1000
    choice_data_subset['Engine effect (kW)'] = choice_data_subset['Engine effect (kW)'] / 100

    variables = ['Prices (2015-DKK)', 'Weight (kg)', 'Engine effect (kW)', 'Size (m3)', 'Cost/km (DKK)']
    N = choice_data_subset.Year.nunique()
    J = choice_data_subset[['Year', 'key']].groupby('Year').count().min().tolist()[0]
    x, y, x_vars = cle.get_x_y(dataset = choice_data_subset, N = N, J = J,  dummyvar = 'Fuel', attributes = variables)

    result = cle.clogt_estimation(x = x, y = y, x_vars = x_vars)
    theta_hat = result['theta_hat']
    
    # CP for de originale priser:
    cp_original_prices = logit.choice_prob(theta_hat, x)
    cp_original_prices = cp_original_prices.reshape(N*J,)
    choice_data_subset = choice_data_subset.assign(cp_original_prices = cp_original_prices)
    cp_original_prices_data = choice_data_subset[['Year', 'cp_original_prices', 'Fuel']].set_index(['Fuel'])
    cp_original_prices_el_data = cp_original_prices_data.loc['El']
    cp_original_prices_el_sum = cp_original_prices_el_data.groupby('Year')['cp_original_prices'].sum()
    cp_original_prices_benzin_data = cp_original_prices_data.loc['Benzin']
    cp_original_prices_benzin_sum = cp_original_prices_benzin_data.groupby('Year')['cp_original_prices'].sum()
    cp_original_prices_diesel_data = cp_original_prices_data.loc['Diesel']
    cp_original_prices_diesel_sum = cp_original_prices_diesel_data.groupby('Year')['cp_original_prices'].sum()
    
    # CP for new_prices:
    choice_data_subset = choice_data_subset.assign(new_prices = new_prices(rate_1, rate_2, rate_3, reg_beløb_1, reg_beløb_2, bundfradrag, el_bundfradrag_1, el_fradrag_2, indfasning))
    choice_data_subset['new_prices'] = np.log(choice_data_subset['new_prices'] / 100000)

    variables_v2 = ['new_prices', 'Weight (kg)', 'Engine effect (kW)', 'Size (m3)', 'Cost/km (DKK)']
    x_v2, y_v2, x_vars_v2 = cle.get_x_y(dataset = choice_data_subset, N = N, J = J,  dummyvar = 'Fuel', attributes = variables_v2)
    
    cp = logit.choice_prob(theta_hat, x_v2)
    cp = cp.reshape(N*J,)
    choice_data_subset = choice_data_subset.assign(cp = cp)

    cp_data = choice_data_subset[['Year', 'cp', 'Fuel']].set_index(['Fuel'])

    cp_el_data = cp_data.loc['El']
    cp_el_sum = cp_el_data.groupby('Year')['cp'].sum()

    cp_benzin_data = cp_data.loc['Benzin']
    cp_benzin_sum = cp_benzin_data.groupby('Year')['cp'].sum()
    
    cp_diesel_data = cp_data.loc['Diesel']
    cp_diesel_sum = cp_diesel_data.groupby('Year')['cp'].sum()

    print("\nSummen af valgsandsynlighederne for elbiler i 2020:")
    print("For originale priser =", round(cp_original_prices_el_sum[2020], 6))
    print("For nye priser =", round(cp_el_sum[2020], 6))
    
    print("\nSummen af valgsandsynlighederne for benzinbiler i 2020:")
    print("For originale priser =", round(cp_original_prices_benzin_sum[2020], 6))
    print("For nye priser =", round(cp_benzin_sum[2020], 6))
    
    print("\nSummen af valgsandsynlighederne for dieselbiler i 2020:")
    print("For originale priser =", round(cp_original_prices_diesel_sum[2020], 6))
    print("For nye priser =", round(cp_diesel_sum[2020], 6))
        
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    
    ax.set_xlabel('År')
    ax.set_ylabel('Valgsandsynlighed elbiler', color = 'black') 
    line1, = ax.plot(cp_el_data['Year'].drop_duplicates(), cp_el_sum, color = 'g')
    ax.set_ylim([0.0, 0.05])
    ax.grid()

    ax2 = ax.twinx()
    ax2.set_ylabel('Valgsandsynlighed benzin- og dieselbiler', color = 'black') 
    line2, = ax2.plot(cp_benzin_data['Year'].drop_duplicates(), cp_benzin_sum, color = 'r')
    ax2.set_ylim([0.0, 1.0])

    line3, = plt.plot(cp_diesel_data['Year'].drop_duplicates(), cp_diesel_sum, color = 'b')

    el_label = mpatches.Patch(color = 'g', label = 'El')
    benzin_label = mpatches.Patch(color = 'r', label = 'Benzin')
    diesel_label = mpatches.Patch(color = 'b', label = 'Diesel')
    ax.legend(handles = [el_label, benzin_label, diesel_label], loc = 2)
    plt.title("Forudsagt efterspørgsel for alle biltyper")

    line1.set_ydata(cp_el_sum)
    line2.set_ydata(cp_benzin_sum)
    line3.set_ydata(cp_diesel_sum)
    fig.canvas.draw_idle()

#interact(interactive_cp, rate_1 = (0.0, 0.5, 0.01), rate_2 = (0.0, 1.7, 0.01), rate_3 = (0.0, 3.0, 0.01), reg_beløb_1 = (0.0, 77100.0, 100.0), reg_beløb_2 = (0.0, 240000.0, 100.0), bundfradrag = (0.0, 25700.0, 100.0), el_bundfradrag_1 = (137000.0, 170000.0, 100.0), el_fradrag_2 = (0.0, 1700.0, 100.0), indfasning = (0.0, 1.0, 0.1));


In [21]:
def skatteprovenu(car_prices):
    choice_data_subset = data_set(path)
    skatteprovenu_lst = []
    sum_year_registrations = choice_data_subset.groupby('Year')['No. of registrations'].sum()
    aarstal = len(sum_year_registrations)
    J = choice_data_subset[['Year', 'key']].groupby('Year').count()
    for aar in range(0, aarstal):
        I = J.iloc[aar].tolist()[0]
        for i in range(0, I):
            car_i_markedsandele = choice_data_subset.loc[i, 'Markedsandele']
            car_i_skatteindtægt = car_prices.iloc[i] - choice_data_subset.loc[i, 'afgiftspligtig_værdi']
            skatteprovenu = car_i_markedsandele * sum_year_registrations.tolist()[aar] * car_i_skatteindtægt
            skatteprovenu_lst.append(skatteprovenu)
    return skatteprovenu_lst

In [27]:
# Indstiller størrelsen på plottet
plt.rcParams['figure.figsize'] = (8.5, 2.5)
plt.rcParams['figure.dpi'] = 170
mpl.rc('xtick', labelsize = 5) 
mpl.rc('ytick', labelsize = 5)
plt.rcParams.update({'font.size': 5})

def interactive_skatteprovenu(rate_1 = 0.25, rate_2 = 0.85, rate_3 = 1.5, reg_beløb_1 = 65000, reg_beløb_2 = 202200, bundfradrag = 21700, el_bundfradrag_1 = 170000, el_fradrag_2 = 1700, indfasning = 0.40):
    choice_data_subset = data_set(path)
    
    skatteprovenu_original_prices = skatteprovenu(choice_data_subset['Prices (2015-DKK)'])
    choice_data_subset = choice_data_subset.assign(skatteprovenu_original_prices = skatteprovenu_original_prices)
    #print("\nOriginal prices:\n", choice_data_subset['Prices (2015-DKK)'])
    #print("\nSkatteprovenu for original prices:\n", choice_data_subset['skatteprovenu_original_prices'])
    
    choice_data_subset = choice_data_subset.assign(new_prices = new_prices(rate_1, rate_2, rate_3, reg_beløb_1, reg_beløb_2, bundfradrag, el_bundfradrag_1, el_fradrag_2, indfasning))
    skatteprovenu_new_prices = skatteprovenu(choice_data_subset['new_prices'])
    choice_data_subset = choice_data_subset.assign(skatteprovenu_new_prices = skatteprovenu_new_prices)
    #print("\nNew prices:\n", choice_data_subset['new_prices'])
    #print("\nSkatteprovenu for new prices:\n", choice_data_subset['skatteprovenu_new_prices'].sum())
    
    skatteprovenu_new_prices_data = choice_data_subset[['Year', 'skatteprovenu_new_prices', 'Fuel']].set_index(['Fuel'])
    skatteprovenu_original_prices_data = choice_data_subset[['Year', 'skatteprovenu_original_prices', 'Fuel']].set_index(['Fuel'])

    # Skatteprovenu for new_prices of benzin
    skatteprovenu_new_prices_benzin_data = skatteprovenu_new_prices_data.loc['Benzin']
    skatteprovenu_new_prices_benzin_sum = skatteprovenu_new_prices_benzin_data.groupby('Year')['skatteprovenu_new_prices'].sum() / 1000000
    skatteprovenu_original_prices_benzin_data = skatteprovenu_original_prices_data.loc['Benzin']
    skatteprovenu_original_prices_benzin_sum = skatteprovenu_original_prices_benzin_data.groupby('Year')['skatteprovenu_original_prices'].sum() / 1000000
    
    # Skatteprovenu for new_prices of diesel
    skatteprovenu_new_prices_diesel_data = skatteprovenu_new_prices_data.loc['Diesel']
    skatteprovenu_new_prices_diesel_sum = skatteprovenu_new_prices_diesel_data.groupby('Year')['skatteprovenu_new_prices'].sum() / 1000000
    skatteprovenu_original_prices_diesel_data = skatteprovenu_original_prices_data.loc['Diesel']
    skatteprovenu_original_prices_diesel_sum = skatteprovenu_original_prices_diesel_data.groupby('Year')['skatteprovenu_original_prices'].sum() / 1000000
    
    # Skatteprovenu for new_prices of el
    skatteprovenu_new_prices_el_data = skatteprovenu_new_prices_data.loc['El']
    skatteprovenu_new_prices_el_sum = skatteprovenu_new_prices_el_data.groupby('Year')['skatteprovenu_new_prices'].sum() / 1000000
    skatteprovenu_original_prices_el_data = skatteprovenu_original_prices_data.loc['El']
    skatteprovenu_original_prices_el_sum = skatteprovenu_original_prices_el_data.groupby('Year')['skatteprovenu_original_prices'].sum() / 1000000
    
    print("\nSummen af skatteprovenuet for elbiler i 2020:")
    print("For originale priser =", round(skatteprovenu_original_prices_el_sum[2020], 6))
    print("For nye priser =", round(skatteprovenu_new_prices_el_sum[2020], 6))
    
    print("\nSummen af skatteprovenuet for benzinbiler i 2020:")
    print("For originale priser =", round(skatteprovenu_original_prices_benzin_sum[2020], 6))
    print("For nye priser =", round(skatteprovenu_new_prices_benzin_sum[2020], 6))
    
    print("\nSummen af skatteprovenuet for dieselbiler i 2020:")
    print("For originale priser =", round(skatteprovenu_original_prices_diesel_sum[2020], 6))
    print("For nye priser =", round(skatteprovenu_new_prices_diesel_sum[2020], 6))
    
    fig, (ax_el, ax_benzin, ax_diesel) = plt.subplots(1, 3)
    
    # Plotting el
    ax_el.set_xlabel('År')
    ax_el.set_ylabel('1 mio. DKK', color = 'black')
    ax_el.grid()
    ax_el.ticklabel_format(style = 'plain')
    
    line_el_1, = ax_el.plot(skatteprovenu_new_prices_el_data['Year'].drop_duplicates(), skatteprovenu_new_prices_el_sum, color = 'y')
    line_el_2, = ax_el.plot(skatteprovenu_original_prices_el_data['Year'].drop_duplicates(), skatteprovenu_original_prices_el_sum, color = 'g', linestyle = 'dashed')
    
    el_nye_priser = mpatches.Patch(color = 'y', label = 'Nye priser')
    el_originale_priser = mpatches.Patch(color = 'g', label = 'Originale priser')
    ax_el.legend(handles = [el_nye_priser, el_originale_priser], loc = 2)
    ax_el.set_title("Samlet årligt skatteprovenu for elbiler")
    
    line_el_1.set_ydata(skatteprovenu_new_prices_el_sum)
    line_el_2.set_ydata(skatteprovenu_original_prices_el_sum)
    fig.canvas.draw_idle()
    
    # Plotting benzin
    ax_benzin.set_xlabel('År')
    #ax_benzin.set_ylabel('1 mio. DKK', color = 'black')
    ax_benzin.grid()
    ax_benzin.ticklabel_format(style = 'plain')
    
    line_benzin_1, = ax_benzin.plot(skatteprovenu_new_prices_benzin_data['Year'].drop_duplicates(), skatteprovenu_new_prices_benzin_sum, color = 'y')
    line_benzin_2, = ax_benzin.plot(skatteprovenu_original_prices_benzin_data['Year'].drop_duplicates(), skatteprovenu_original_prices_benzin_sum, color = 'r', linestyle = 'dashed')
    
    benzin_nye_priser = mpatches.Patch(color = 'y', label = 'Nye priser')
    benzin_originale_priser = mpatches.Patch(color = 'r', label = 'Originale priser')
    ax_benzin.legend(handles = [benzin_nye_priser, benzin_originale_priser], loc = 2)
    ax_benzin.set_title("Samlet årligt skatteprovenu for benzinbiler")

    line_benzin_1.set_ydata(skatteprovenu_new_prices_benzin_sum)
    line_benzin_2.set_ydata(skatteprovenu_original_prices_benzin_sum)
    fig.canvas.draw_idle()

    # Plotting diesel
    ax_diesel.set_xlabel('År')
    #ax_diesel.set_ylabel('1 mio. DKK', color = 'black')
    ax_diesel.grid()
    ax_diesel.ticklabel_format(style = 'plain')
    
    line_diesel_1, = ax_diesel.plot(skatteprovenu_new_prices_diesel_data['Year'].drop_duplicates(), skatteprovenu_new_prices_diesel_sum, color = 'y')
    line_diesel_2, = ax_diesel.plot(skatteprovenu_original_prices_diesel_data['Year'].drop_duplicates(), skatteprovenu_original_prices_diesel_sum, color = 'b', linestyle = 'dashed')
    
    diesel_nye_priser = mpatches.Patch(color = 'y', label = 'Nye priser')
    diesel_originale_priser = mpatches.Patch(color = 'b', label = 'Originale priser')
    ax_diesel.legend(handles = [diesel_nye_priser, diesel_originale_priser], loc = 2)
    ax_diesel.set_title("Samlet årligt skatteprovenu for dieselbiler")
    
    line_diesel_1.set_ydata(skatteprovenu_new_prices_diesel_sum)
    line_diesel_2.set_ydata(skatteprovenu_original_prices_diesel_sum)
    fig.canvas.draw_idle()
    
interact(interactive_skatteprovenu, rate_1 = (0.0, 0.5, 0.01), rate_2 = (0.0, 1.7, 0.01), rate_3 = (0.0, 3.0, 0.01), reg_beløb_1 = (0.0, 77100.0, 100.0), reg_beløb_2 = (0.0, 240000.0, 100.0), bundfradrag = (0.0, 25700.0, 100.0), el_bundfradrag_1 = (137000.0, 170000.0, 100.0), el_fradrag_2 = (0.0, 1700.0, 100.0), indfasning = (0.0, 1.0, 0.1));


interactive(children=(FloatSlider(value=0.25, description='rate_1', max=0.5, step=0.01), FloatSlider(value=0.8…